# Large, three-generation CEPH families reveal post-zygotic mosaicism and variability in germline mutation accumulation

### Thomas A. Sasani, Brent S. Pedersen, Ziyue Gao, Lisa M. Baird, Molly Przeworski, Lynn B. Jorde, Aaron R. Quinlan

### Identifying inter-family variablity in parental age effects

Using the full callset of *de novo* variants in the F2 generation, our goal is to see if we can improve the fit of a model that tries to predict `autosomal_dnms` as a function of `dad_age` by taking family into account.

To do this, we created a summary dataframe containing the total number of DNMs in each individual, the individual's paternal and maternal ages at conception, and the family from which the individual was sampled.

In [ ]:
f2 = read.csv("../data/f2.dnms.summary.csv")

head(f2)

We first fit a Poisson regression as follows:

In [ ]:
model_a = glm(autosomal_dnms ~ dad_age, data=f2, family=poisson(link="identity"))

We find a highly significant effect of paternal age on total DNM burden (as expected).

In [ ]:
print(anova(model_a, test="Chisq"))
summary(model_a)

But since this model is agnostic to the family from which each F2 individual was “sampled,” we fit an additional model that incorporated the effects of family on the paternal age effect.

In [ ]:
model_b = glm(autosomal_dnms ~ dad_age * family_id, data=f2, family=poisson(link="identity"))

We ran an ANOVA on this fitted model, and find that the additive `family` term is highly significant. Adding the `dad_age:family` interaction term produces a significantly better fit, as well, suggesting that variability in regression intercepts and slopes between CEPH families is important. 


In [ ]:
print(anova(model_b, test="Chisq"))

Additionally, simply comparing the "family-aware" and "family-agnostic" models suggests that the former is a much better fit.

In [ ]:
print(anova(model_b, model_a, test="Chisq"))

### Accounting for the effects of both paternal and maternal age

To account for the effect of maternal age on *de novo* mutation burden, we fit a model that incorporated the effects of both maternal and paternal age, as well as family ID. 

In [ ]:
model_c = glm(autosomal_dnms ~ dad_age + mom_age + family_id, data=f2, family=poisson(link="identity"))

And after running an ANOVA on this model, we find that including the `family_id` term produces a significantly better fit than including `dad_age` and `mom_age` alone.

In [ ]:
print(anova(model_c, test="Chisq"))

### Accounting for the effects of variable sequencing coverage

It's also possible that variability in the "callable fraction" of the genome across F2s could impact these results, since this callable fraction will directly impact the total number of DNMs we call in each sample. Therefore, we fit a model that takes this (autosomal) callable fraction into account.

Before fitting this model, let's first consider how we're modeling the number DNMs in each F2 individual. As our model is only considering the effects of paternal age on total DNM burden, the mutation rate $\mu$ can be defined as:

$\mu  = \beta_p A_p + \beta_0$

Where $\beta_p$ is the paternal age effect, $A_p$ is paternal age, and $\beta_0$ is an intercept term.

Therefore, the number of DNMs in a sample is assumed to follow a Poisson distribution, with the expected mean of the distribution defined as:

$E (\# DNMs) = \mu C$

Where $C$ is the callable fraction of the genome.

Thus,

$E (\# DNMs) = (\beta_p A_p + \beta_0) C$

And if we scale $A_p$ by the callable fraction and call this new term $A_{p\_scaled}$, then:

$E (\# DNMs) = \beta_p A_{p\_scaled} + \beta_0 C$

Following this, we can fit a null model that takes into account each sample's callable fraction as follows:

In [ ]:
# generate the `dad_age_scaled` term
f2$dad_age_scaled = f2$dad_age * f2$autosomal_callable_fraction

model_c_null = glm(autosomal_dnms ~ dad_age_scaled + autosomal_callable_fraction + 0, data=f2, family=poisson(link="identity"))

And we can then compare a model that takes both the callable fraction **and** family into account to this null model.

In [ ]:
model_c_exp = glm(autosomal_dnms ~ dad_age_scaled * family_id + autosomal_callable_fraction + 0, data=f2, family=poisson(link="identity"))

In [ ]:
print(anova(model_c_null, model_c_exp, test="Chisq"))